# MP2 二阶梯度：轨道旋转途径

这份文档将会从另一个方式实现 MP2 二阶梯度．这种方式是“轨道旋转”方式，其在 Handy, N. C. et al. CPL 120, 151 (doi: 10.1016/0009-2614(85)87031-7) 有所描述．这里就完全按照 Handy 文章的思路进行程序化．

需要注意的是，这篇文档暂时还没有打算将微分 Z-Vector 方程应用于轨道旋转途径．因此，这里的实现方案是相当低效的．

In [1]:
import numpy as np
from pyscf import scf, gto, lib, grad, hessian, dft, mp
import pyscf.hessian.rks
import pyscf.grad.rks
from functools import partial
import pickle

from utilities import val_from_fchk, NumericDiff
from hessian import HFHelper, GGAHelper

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.einsum_path = partial(np.einsum_path, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(6, linewidth=150, suppress=True)

In [2]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

In [3]:
# We only U_1 in this document
# Pre-generate some code here will make later work quicker
# But these operations or memory consuming
hfh = HFHelper(mol)
hfh.U_1_vo
hfh.eri0_mo
hfh.eri1_mo
hfh.eri2_mo
hfh

/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:229: UserWarning: eri0_mo: ERI AO -> MO is quite expensive!
  warnings.warn("eri0_mo: ERI AO -> MO is quite expensive!")
/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:222: UserWarning: eri0_ao: ERI should not be stored in memory! Consider J/K engines!
  warnings.warn("eri0_ao: ERI should not be stored in memory! Consider J/K engines!")
/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:279: UserWarning: eri1_mo: 4-idx tensor ERI should be not used!
  warnings.warn("eri1_mo: 4-idx tensor ERI should be not used!")
/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:272: UserWarning: eri1_ao: 4-idx tensor ERI should be not used!
  warnings.warn("eri1_ao: 4-idx tensor ERI should be not used!")
/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:317: UserWarning: eri2_mo: 4-idx tensor ERI should be not used!
  warnings.warn("eri2_mo: 4-idx tensor ERI should be not used

## MP2 二阶梯度：参考值

我们在以前的代码中讨论过在与 MP2 Hessian 相关的 [资源文件夹](include/mp2_hess) 参考值：

In [4]:
d_hess = mol.natm * 3
g_array = val_from_fchk("Cartesian Force Constants", "include/mp2_hess/mp2_hess.fchk")
hess_mp2_gaussian = np.zeros((d_hess, d_hess))
p = 0
for d1 in range(d_hess):
    for d2 in range(d1 + 1):
        hess_mp2_gaussian[d1][d2] = hess_mp2_gaussian[d2][d1] = g_array[p]
        p += 1
hess_mp2_gaussian = hess_mp2_gaussian.reshape((natm, 3, natm, 3)).swapaxes(1, 2)

In [5]:
hess_mp2_ref = hess_mp2_gaussian - hfh.scf_hess.kernel()

## MP2 一阶梯度：重新表述

在 MP2 二阶梯度前，我们仍然要回顾 MP2 的一阶梯度。我们会使用 Handy 提供的方法编写 MP2 的一阶梯度．我们先需要生成 MP2 一阶梯度的参考值：

### 重要变量定义

In [6]:
mp2_eng = mp.MP2(hfh.scf_eng)
mp2_eng.kernel()[0]
mp2_grad = grad.mp2.Gradients(mp2_eng)
mp2_grad.kernel()
grad_mp2_ref = mp2_grad.de - hfh.scf_grad.kernel()

In [7]:
%%capture

e, eo, ev = hfh.e, hfh.eo, hfh.ev
C, Co, Cv = hfh.C, hfh.Co, hfh.Cv
D = hfh.D
eri0_mo = hfh.eri0_mo
eri1_mo = hfh.eri1_mo
eri2_mo = hfh.eri2_mo

F_1_mo = hfh.F_1_mo
S_1_mo = hfh.S_1_mo
F_2_mo = hfh.F_2_mo
S_2_mo = hfh.S_2_mo
U_1_vo = hfh.U_1_vo
U_1_ov = hfh.U_1_ov
Ax0_Core = hfh.Ax0_Core
Ax1_Core = hfh.Ax1_Core

D_iajb = lib.direct_sum("i - a + j - b", hfh.eo, hfh.ev, hfh.eo, hfh.ev)

### 旋转下的 U 矩阵

对于非占-占据与占据-非占部分的 U 矩阵，其可以通过 CP-HF 方程求解；我们已经在上述代码中，生成了 `U_1_vo` $U_{ai}^{A_t}$ 与 `U_1_ov` $U_{ia}^{A_t}$．但是对于占据-占据、非占-非占部分的 U 矩阵，对于其非对角元，如果在 Canonical HF 下，应当表示为

$$
U_{pq} = - \frac{1}{\varepsilon_p - \varepsilon_q} (B_{pq}^{A_t} + A_{pq, bj} U_{bj}^{A_t}) \quad (p \neq q)
$$

显然，这并不是一个好的解决方案；因为如果基组非常大，或者分子具有能使占据或非占轨道简并的对称性，那么占据-占据和非占-非占 U 矩阵将会出现无穷大．

Handy 指出，由于轨道 (或波函数？其实我还不了解) 旋转的存在 (HF、MP2 等方法的能量是旋转不变的)，使得我们不一定要在 Canonical HF 下进行梯度的求取计算．尽管不是任意的轨道旋转都是允许的，但只要满足以下两个条件即可：只在占据-占据与非占-非占部分旋转 (即 $U_{ai}^{A_t}$ 仍然使用 CP-HF 方程求解)，以及满足 $U_{pq}^{A_t} + U_{pq}^{A_t} = - S_{pq}^{A_t}$ 即可．

既然任意的旋转都是允许的，Handy 在文章中取了一种方法，即对于占据、非占部分，其 `U_1R` U 矩阵表示为

$$
U_{pq}^{A_t} = - \frac{1}{2} S_{pq}^{A_t} \quad (p, q \in \mathrm{occ} || p, q \in \mathrm{vir})
$$

In [8]:
U_1R = - 0.5 * S_1_mo
U_1R[:, :, sv, so] = U_1_vo
U_1R[:, :, so, sv] = - S_1_mo[:, :, so, sv] - U_1_vo.swapaxes(-1, -2)

而所有与一阶 U 矩阵有关的计算全部使用上述方式进行替代．

附带指出，这种旋转对于二阶 CP-HF 方程，我自己还不清楚其原理，因为二阶 U 矩阵的限定条件是

$$
U_{pq}^{A_t B_s} + U_{pq}^{A_t B_s} = - \xi_{pq}^{A_t B_s}
$$

但 $\xi_{pq}^{A_t B_s}$ 本身也是与 $U_{kl}^{A_t}$ 相关的量．因此后面就不讨论二阶 U 矩阵的限定条件，而只是对其进行程序化．

### 重塑表达式

我们在这里先采取

$$
E_\mathrm{elec}^{\mathrm{MP2}} = \tilde g_{ij}^{ab} g_{ij}^{ab} D_{ij}^{ab}
$$

* `g_mo`: $g_{pq}^{rs} = (pq|rs)$

* `G_mo`: $\tilde g_{pq}^{rs} = 2 g_{pq}^{rs} - g_{pq}^{sr}$

* `g_iajb`: $g_{ij}^{ab}$

* `G_iajb`: $\tilde g_{ij}^{ab}$

* `D_iajb`: $D_{ij}^{ab} = \varepsilon_i - \varepsilon_a + \varepsilon_j - \varepsilon_b$

注意这里的 $D_{ij}^{ab}$ 与 Handy (13) 的符号刚好相反．

In [9]:
g_mo = eri0_mo
G_mo = 2 * g_mo - g_mo.swapaxes(-1, -3)
g_iajb = g_mo[so, sv, so, sv]
G_iajb = G_mo[so, sv, so, sv]
D_iajb = lib.direct_sum("i - a + j - b", eo, ev, eo, ev)

In [10]:
np.allclose((g_iajb * G_iajb / D_iajb).sum(), mp2_eng.e_corr)

True

但后续公式中，如果使用 MP2 振幅，在处理不少问题上会简化不少：

* `t_iajb`: $t_{ij}^{ab} = g_{ij}^{ab} (D_{ij}^{ab})^{-1}$

* `T_iajb`: $T_{ij}^{ab} = \tilde g_{ij}^{ab} (D_{ij}^{ab})^{-1}$

In [11]:
t_iajb = g_iajb / D_iajb
T_iajb = 2 * t_iajb - t_iajb.swapaxes(-1, -3)

我们尽管使用 Handy 的策略，但这份文档的整个 MP2 梯度推导过程将仍然是暴力求解而不进行简化．仿照我们以前的直接梯度推导，只是将 U 矩阵换为轨道旋转后的 U 矩阵，可以得到

* `pd_g_mo`: $\partial_{A_t} g_{ij}^{ab} = (ia|jb)^{A_t}$ (全轨道)

* `pdRU_g_mo`: $\partial_{A_t}^\mathrm{U} g_{ij}^{ab} = g_{pj}^{ab} U_{pi}^{A_t} + g_{ip}^{ab} U_{pj}^{A_t} + g_{ij}^{pb} U_{pa}^{A_t} + g_{ij}^{ap} U_{pb}^{A_t}$ (全轨道)

* `pdR_g_mo`: $g_{ij}^{ab, (A_t)} = \frac{\partial}{\partial A_t} g_{ij}^{ab} = (\partial_{A_t} + \partial_{A_t}^\mathrm{U}) g_{ij}^{ab} = (ia|jb)^{A_t}$ (全轨道)

* `pdR_G_mo`: $\tilde g_{ij}^{ab, (A_t)} = \frac{\partial}{\partial A_t} \tilde g_{ij}^{ab} = 2 \frac{\partial}{\partial A_t} g_{ij}^{ab} - \frac{\partial}{\partial A_t} g_{ij}^{ba}$ (全轨道)

以及占据-非占-占据-非占的分割 `pdA_g_iajb`, `pdA_G_iajb`：

In [12]:
pd_g_mo = eri1_mo
pdRU_g_mo = (
    + np.einsum("pjkl, Atpi -> Atijkl", g_mo, U_1R)
    + np.einsum("ipkl, Atpj -> Atijkl", g_mo, U_1R)
    + np.einsum("ijpl, Atpk -> Atijkl", g_mo, U_1R)
    + np.einsum("ijkp, Atpl -> Atijkl", g_mo, U_1R)
)
pdR_g_mo = pd_g_mo + pdRU_g_mo

pdR_G_mo = 2 * pdR_g_mo - pdR_g_mo.swapaxes(-1, -3)

pdR_g_iajb = pdR_g_mo[:, :, so, sv, so, sv]
pdR_G_iajb = pdR_G_mo[:, :, so, sv, so, sv]

但不像以前一样会直接生成 $\frac{\partial}{\partial A_t} D_{ij}^{ab}$，因为轨道旋转过后，$\frac{\partial}{\partial A_t} \varepsilon_p$ 的结果并不是带原子分量方向的一维向量．取而代之的，是我们要显式地写出对 Fock 矩阵的全导数 `F_1R` $\frac{\partial}{\partial A_t} F_{pq}$．但这样的记号比较繁琐；为了简便，这里将采取与 Handy 类似的记号，`F_1R` $F_{pq}^{(A_t)}$：

Yamaguchi (p433 W.1); Handy (7)

$$
F_{pq}^{(A_t)} = F_{pq}^{A_t} + U_{pq}^{A_t} \varepsilon_p + U_{qp}^{A_t} \varepsilon_q + A_{pq, bj} U_{bj}^{A_t} - \frac{1}{2} A_{pq, kl} S_{kl}^{A_t}
$$

In [13]:
F_1R = (
    + F_1_mo
    + np.einsum("Atpq, p -> Atpq", U_1R, e)
    + np.einsum("Atqp, q -> Atpq", U_1R, e)
    + Ax0_Core(sa, sa, sv, so)(U_1_vo)
    - 0.5 * Ax0_Core(sa, sa, so, so)(S_1_mo[:, :, so, so])
)

原则上，我们可以验证上述矩阵的三个性质：

* 所有非占-占据和占据-非占部分均为零；

* 对角线元素恰是 Canonical HF 图景下轨道能全导数；

* 若 U 矩阵采用 Canonical HF 图景，那么上述张量就是对角的．

因此，MP2 的梯度可以写作以下表达式：

$$
E_\mathrm{elec}^{\mathrm{MP2}, A_t} =
2 T_{ij}^{ab} (g_{ij}^{ab})^{(A_t)}
+ 2 T_{ij}^{ab} t_{ij}^{ac} F_{bc}^{(A_t)}
- 2 T_{ij}^{ab} t_{ik}^{ab} F_{jk}^{(A_t)}
$$

In [14]:
np.allclose(
    + 2 * np.einsum("iajb, Atiajb -> At", T_iajb, pdR_g_mo[:, :, so, sv, so, sv])
    + 2 * np.einsum("iajb, iajc, Atbc -> At", T_iajb, t_iajb, F_1R[:, :, sv, sv])
    - 2 * np.einsum("iajb, iakb, Atjk -> At", T_iajb, t_iajb, F_1R[:, :, so, so])
    ,
    grad_mp2_ref
)

True

<div class="alert alert-warning">

**提醒**

需要注意的是，Handy (11) 的公式很可能是错误的；这个公式应该重新写为

$$
E_2^{(X)} = - 2 \sum_{ij} \sum_{ab} (ia|jb)^{(X)} t_{ij}^{ab} + 2 \sum_{bc} V_{bc} \epsilon_{bc}^{(X)} - 2 \sum_{jk} V_jk \epsilon_{jk}^{(X)}
$$

</div>

## MP2 二阶梯度：直接求导法

下面我们会通过暴力求解的方法直接得到 MP2 的二阶梯度．所谓暴力求解指的是依据方才得到的 $U_{pq}^{A_t}$ 求解必要项，包括 ERI 梯度、一阶 U 矩阵等的全导数等．

### 旋转后的二阶 U 矩阵

尽管不清楚原理，但旋转后的二阶 U 矩阵可以通过下述方法生成：对于其非占-占据和占据-非占部分，通过二阶 CP-HF 方程获取；对于其占据-占据和非占-非占部分，通过旋转的条件给出：

$$
U_{pq}^{A_t B_s} + U_{pq}^{A_t B_s} = - \xi_{pq}^{A_t B_s}
$$

不管是 CP-HF 方程还是生成占据-占据和非占-非占的二阶 U 矩阵的生成，都需要 `Xi_2R` $\xi_{pq}^{A_t B_s}$．尽管生成该项的公式相同，但我们使用了 `U_1R` 旋转后的 U 矩阵，而非 `hfh.U_1` Canonical HF 的 U 矩阵．因此，我们必须在这里重新生成 $\xi_{pq}^{A_t B_s}$，而不能采用 Canonical HF 的 `hfh.Xi_2`：

Yamaguchi (p405 L.4); Handy (26)

$$
\xi_{pq}^{A_t B_s} = S_{pq}^{A_t B_s} + U_{pm}^{A_t} U_{qm}^{B_s} + U_{pm}^{B_s} U_{qm}^{A_t} - S_{pm}^{A_t} S_{qm}^{B_s} + S_{pm}^{B_s} S_{qm}^{A_t}
$$

In [15]:
Xi_2R = (
    hfh.S_2_mo
    + np.einsum("Atpm, Bsqm -> ABtspq", U_1R, U_1R)
    + np.einsum("Bspm, Atqm -> ABtspq", U_1R, U_1R)
    - np.einsum("Atpm, Bsqm -> ABtspq", S_1_mo, S_1_mo)
    - np.einsum("Bspm, Atqm -> ABtspq", S_1_mo, S_1_mo)
)

随后我们生成二阶 U 矩阵的非占-占据部分．其 CP-HF 方程的等式右是 `B_2R` $B_{ai}^{A_t B_s}$；但我们以后还要生成 `F_2R` $F_{pq}^{(A_t B_s)} = \frac{\partial^2}{\partial A_t \partial B_s} F_{pq}$，在这里不妨生成全轨道的 $B_{pq}^{A_t B_s}$．

对于生成 $B_{pq}^{A_t B_s}$ 的过程，Handy 和 Yamaguchi 有不同的过程；但这两个过程给出的结果是相同的．我们把 Handy 生成的 $B_{pq}^{A_t B_s}$ 命名为 `B_2R`，而把 Yamaguchi 生成的命名为 `B_2R_`．那么，

Handy (36)，注意 Handy 文章里的 CP-HF 方程的符号与 Yamaguchi 的符号相反，我们采用 Yamaguchi 的符号：

\begin{align}
B_{ai}^{A_t B_s} =
&+ \frac{1}{2} F_{ai}^{A_t B_s} - \frac{1}{2} \xi_{ai}^{A_t B_s} \varepsilon_i - \frac{1}{4} A_{ai, kl} \xi_{kl}^{A_t B_s} + A_{ai, pk}^{A_t} U_{pk}^{B_s} - \frac{1}{2} A_{ai, pq} U_{pk}^{A_t} U_{qk}^{B_s} \\
&+ U_{pa}^{A_t} F_{pi}^{(B_s)} + U_{pi}^{A_t} F_{pa}^{B_s} \\
&- U_{pa}^{A_t} U_{ip}^{B_s} \varepsilon_i - U_{pi}^{A_t} U_{pa}^{B_s} \varepsilon_p - U_{pi}^{A_t} U_{ap}^{B_s} \varepsilon_a \\
&+ \mathrm{interchange} (A_t, B_s)
\end{align}

In [16]:
B_2R = (
    # line 1
    0.5 * hfh.F_2_mo
    - 0.5 * np.einsum("ABtsai, i -> ABtsai", Xi_2R, e)
    - 0.25 * Ax0_Core(sa, sa, so, so)(Xi_2R[:, :, :, :, so, so])
    + Ax1_Core(sa, sa, sa, so)(U_1R[:, :, :, so])
    + 0.5 * Ax0_Core(sa, sa, sa, sa)(np.einsum("Atkm, Bslm -> ABtskl", U_1R[:, :, :, so], U_1R[:, :, :, so]))
    # line 2
    + np.einsum("Atpa, Bspi -> ABtsai", U_1R, F_1R)
    + np.einsum("Atpi, Bspa -> ABtsai", U_1R, F_1R)
    # line 3
    - np.einsum("Atpa, Bsip, i -> ABtsai", U_1R, U_1R, e)
    - np.einsum("Atpi, Bspa, p -> ABtsai", U_1R, U_1R, e)
    - np.einsum("Atpi, Bsap, a -> ABtsai", U_1R, U_1R, e)
)
B_2R += B_2R.transpose((1, 0, 3, 2, 4, 5))

Yamaguchi (p437-438, X.5)

\begin{align}
B_{ai}^{A_t B_s} = 
&+ \frac{1}{2} F_{ai}^{A_t B_s} - \frac{1}{2} \xi_{ai}^{A_t B_s} \varepsilon_i - \frac{1}{4} A_{ai, kl} \xi_{kl}^{A_t B_s} \\
&+ U_{pa}^{A_t} F_{pi}^{B_s} + U_{pi}^{A_t} F_{pa}^{B_s} \\
&+ U_{pa}^{A_t} U_{pi}^{B_s} \varepsilon_p \\
&+ \frac{1}{2} A_{ai, pq} U_{pk}^{A_t} U_{qk}^{B_s} \\
&+ U_{pa}^{A_t} A_{pi, qm} U_{qm}^{B_s} \\
&+ U_{pi}^{A_t} A_{pa, qm} U_{qm}^{B_s} \\
&+ A_{ai, pk}^{A_t} U_{pk}^{B_s} \\
&+ \mathrm{interchange} (A_t B_s)
\end{align}

In [17]:
B_2R_ = (
    # line 1
    0.5 * hfh.F_2_mo
    - 0.5 * np.einsum("ABtsai, i -> ABtsai", Xi_2R, e)
    - 0.25 * Ax0_Core(sa, sa, so, so)(Xi_2R[:, :, :, :, so, so])
    # line 2
    + np.einsum("Atpa, Bspi -> ABtsai", U_1R, F_1_mo)
    + np.einsum("Atpi, Bspa -> ABtsai", U_1R, F_1_mo)
    # line 3
    + np.einsum("Atpa, Bspi, p -> ABtsai", U_1R, U_1R, e)
    # line 4
    + 0.5 * Ax0_Core(sa, sa, sa, sa)(np.einsum("Atkm, Bslm -> ABtskl", U_1R[:, :, :, so], U_1R[:, :, :, so]))
    # line 5
    + np.einsum("Atpa, Bspi -> ABtsai", U_1R, Ax0_Core(sa, sa, sa, so)(U_1R[:, :, :, so]))
    # line 6
    + np.einsum("Atpi, Bspa -> ABtsai", U_1R, Ax0_Core(sa, sa, sa, so)(U_1R[:, :, :, so]))
    # line 7
    + Ax1_Core(sa, sa, sa, so)(U_1R[:, :, :, so])
)
B_2R_ += B_2R_.transpose((1, 0, 3, 2, 4, 5))

In [18]:
np.allclose(B_2R, B_2R_)

True

下面就是正常地代入 CP-HF 求解的框架，求解 `U_2R_vo` $U_{ai}^{A_t B_s}$：

Yamaguchi (p437, X.4)

$$
- (\varepsilon_a - \varepsilon_i) U_{ai}^{A_t B_s} - A_{ai, bj} U_{bj}^{A_t B_s} = B_{pq}^{A_t B_s}
$$

In [19]:
U_2R_vo = scf.cphf.solve(
    Ax0_Core(sv, so, sv, so),
    e,
    hfh.mo_occ,
    B_2R[:, :, :, :, sv, so].reshape(-1, nvir, nocc),
    max_cycle=100,
    tol=1e-11,
    hermi=False
)[0]
U_2R_vo.shape = (natm, natm, 3, 3, nvir, nocc)

最后，根据 Handy (27) 的指示，将 `U_2R` $U_{pq}^{A_t B_s}$ 的剩余部分填补完整：

Handy (27)

$$
U_{pq}^{A_t B_s} = - \frac{1}{2} \xi_{pq}^{A_t} \quad (p, q \in \mathrm{occ} || p, q \in \mathrm{vir})
$$

In [20]:
U_2R = - 0.5 * Xi_2R
U_2R[:, :, :, :, sv, so] = U_2R_vo
U_2R[:, :, :, :, so, sv] = - Xi_2R[:, :, :, :, so, sv] - U_2R_vo.swapaxes(-1, -2)

在获得旋转后的二阶 U 矩阵后，我们可以立即求取 `F_2R` $F_{pq}^{(A_t B_s)} = \frac{\partial^2}{\partial A_t \partial B_s} F_{pq}$．尽管在 Yamaguchi (p433-p434, W.1) 中有求解过程，但这里我们采用更为方便的导出方式：

Handy (34)

$$
F_{pq}^{(A_t B_s)} = B_{pq}^{A_t B_s} A_{pq, bj} U_{bj}^{A_t B_s} + U_{pq}^{A_t B_s} (\varepsilon_p - \varepsilon_q)
$$

In [21]:
F_2R = (
    + B_2R
    + Ax0_Core(sa, sa, sv, so)(U_2R_vo)
    + np.einsum("ABtspq, p -> ABtspq", U_2R, e)
    - np.einsum("ABtspq, q -> ABtspq", U_2R, e)
)

上述张量仍然具有非占-占据和占据-非占部分的值是零的性质，但其迹已经不是 Canonical HF 下的轨道能二阶导数了．

### 轨道旋转后的双电子积分

在生成二阶 U 矩阵后，我们可以求取双电子积分的二阶全导数了．这与以前的做法没有实质区别，只是将所有出现 U 矩阵的地方替换为轨道旋转后的 U 矩阵．

尽管下面的代码不短，但真正有用的变量只有以下一项：

* `pdR_pdR_g_iajb` $g_{ij}^{ab, (A_t B_s)} = \frac{\partial^2}{\partial A_t \partial B_s} g_{ij}^{ab}$

In [22]:
pdRU_U_1 = U_2R - np.einsum("Atpm, Bsmq -> ABtspq", U_1R, U_1R)

pd_pd_g_mo = eri2_mo

pdRU_pd_g_mo = (
    # pd on g
    + np.einsum("Atpjkl, Bspi -> ABtsijkl", pd_g_mo, U_1R)
    + np.einsum("Atipkl, Bspj -> ABtsijkl", pd_g_mo, U_1R)
    + np.einsum("Atijpl, Bspk -> ABtsijkl", pd_g_mo, U_1R)
    + np.einsum("Atijkp, Bspl -> ABtsijkl", pd_g_mo, U_1R)
)
pd_pdRU_g_mo = pdRU_pd_g_mo.swapaxes(0, 1).swapaxes(2, 3)

pdRU_pdRU_g_mo = (
    # pdU on g * U
    + np.einsum("Atpjkl, Bspi -> ABtsijkl", pdRU_g_mo, U_1R)
    + np.einsum("Atipkl, Bspj -> ABtsijkl", pdRU_g_mo, U_1R)
    + np.einsum("Atijpl, Bspk -> ABtsijkl", pdRU_g_mo, U_1R)
    + np.einsum("Atijkp, Bspl -> ABtsijkl", pdRU_g_mo, U_1R)
    # g * pdU on U
    + np.einsum("pjkl, ABtspi -> ABtsijkl", g_mo, pdRU_U_1)
    + np.einsum("ipkl, ABtspj -> ABtsijkl", g_mo, pdRU_U_1)
    + np.einsum("ijpl, ABtspk -> ABtsijkl", g_mo, pdRU_U_1)
    + np.einsum("ijkp, ABtspl -> ABtsijkl", g_mo, pdRU_U_1)
)

pdR_pdR_g_mo = pd_pd_g_mo + pd_pdRU_g_mo + pdRU_pd_g_mo + pdRU_pdRU_g_mo
pdR_pdR_g_iajb = pdR_pdR_g_mo[:, :, :, :, so, sv, so, sv]

### MP2 二阶梯度结果

最后，我们依照 Handy (23, 28-32) 的公式，给出 MP2 二阶梯度的最终结果．注意 Handy 的 $D_{ij}^{ab}$ 的符号与 Bartlett, Aikens 等人的正负号不一致，我们在正负号上采用后者；因此在 $D_{ij}^{ab}$ 与 $t_{ij}^{ab}, T_{ij}^{ab}$ 出现奇数次的正负号取了相反值．

这里推测，对 MP2 能量的求导可以通过 ASG 图形来解决．对于 Handy (32)，可以采用在 MP2 能量中添加两个单电子算符来验证其公式的正确性．

Handy (28)

$$
2 \frac{\partial^2 g_{ij}^{ab}}{\partial A_t \partial B_s} T_{ij}^{ab}
$$

Handy (29)

$$
2 \frac{\partial g_{ij}^{ab}}{\partial A_t} \frac{\partial \tilde g_{ij}^{ab}}{\partial B_s} \frac{1}{D_{ij}^{ab}}
$$

Handy (30)

$$
4 \frac{\partial g_{ij}^{ab}}{\partial A_t} (T_{ij}^{ac} F_{bc}^{(B_s)} - T_{ik}^{ab} F_{jk}^{(B_s)}) \frac{1}{D_{ij}^{ab}}
$$

Handy (31)

$$
2 g_{ij}^{ab} (T_{ij}^{ac} F_{bc}^{(A_t B_s)} - T_{ik}^{ab} F_{jk}^{(A_t B_s)}) \frac{1}{D_{ij}^{ab}}
$$

Handy (32)

\begin{align}
&
+ 4 t_{ij}^{ab} T_{ij}^{cd} (D_{ij}^{cb})^{-1} F_{ac}^{(A_t)} F_{bd}^{(B_s)}
+ 4 t_{ij}^{ab} T_{ij}^{db} (D_{ij}^{cb})^{-1} F_{ac}^{(A_t)} F_{cd}^{(B_s)}
\\ &
+ 4 t_{ij}^{ab} T_{kl}^{ab} (D_{jk}^{ab})^{-1} F_{ki}^{(A_t)} F_{lj}^{(B_s)}
+ 4 t_{ij}^{ab} T_{lj}^{ab} (D_{jk}^{ab})^{-1} F_{ki}^{(A_t)} F_{lk}^{(B_s)}
\\ &
- 4 t_{ij}^{ab} T_{jk}^{bd} (D_{jk}^{ab})^{-1} F_{ki}^{(A_t)} F_{ad}^{(B_s)}
- 4 t_{ij}^{ab} T_{jk}^{bd} (D_{ij}^{db})^{-1} F_{ad}^{(A_t)} F_{ki}^{(B_s)}
\\ &
- 4 t_{ij}^{ab} T_{jk}^{da} (D_{jk}^{ab})^{-1} F_{ki}^{(A_t)} F_{bd}^{(B_s)}
- 4 t_{ij}^{ab} T_{jk}^{da} (D_{ij}^{da})^{-1} F_{bd}^{(A_t)} F_{ki}^{(B_s)}
\end{align}

In [23]:
hess_mp2 = (
    # Handy (28)
    + 2 * np.einsum("ABtsiajb, iajb -> ABts", pdR_pdR_g_iajb, T_iajb)
    # Handy (29)
    + 2 * np.einsum("Atiajb, Bsiajb, iajb -> ABts", pdR_g_iajb, pdR_G_iajb, 1 / D_iajb)
    # Handy (30)
    + 4 * np.einsum("Atiajb, iajc, Bsbc, iajb -> ABts", pdR_g_iajb, T_iajb, F_1R[:, :, sv, sv], 1 / D_iajb)
    - 4 * np.einsum("Atiajb, iakb, Bsjk, iajb -> ABts", pdR_g_iajb, T_iajb, F_1R[:, :, so, so], 1 / D_iajb)
    # Handy (30), interchange A_t and B_s
    + 4 * np.einsum("Bsiajb, iajc, Atbc, iajb -> ABts", pdR_g_iajb, T_iajb, F_1R[:, :, sv, sv], 1 / D_iajb)
    - 4 * np.einsum("Bsiajb, iakb, Atjk, iajb -> ABts", pdR_g_iajb, T_iajb, F_1R[:, :, so, so], 1 / D_iajb)
    # Handy (31)
    + 2 * np.einsum("iajb, iajc, ABtsbc, iajb -> ABts", g_iajb, T_iajb, F_2R[:, :, :, :, sv, sv], 1 / D_iajb)
    - 2 * np.einsum("iajb, iakb, ABtsjk, iajb -> ABts", g_iajb, T_iajb, F_2R[:, :, :, :, so, so], 1 / D_iajb)
    # Handy (32)
    + 4 * np.einsum("iajb, icjd, icjb, Atac, Bsbd -> ABts", t_iajb, T_iajb, 1 / D_iajb, F_1R[:, :, sv, sv], F_1R[:, :, sv, sv])
    + 4 * np.einsum("iajb, idjb, icjb, Atac, Bscd -> ABts", t_iajb, T_iajb, 1 / D_iajb, F_1R[:, :, sv, sv], F_1R[:, :, sv, sv])
    + 4 * np.einsum("iajb, kalb, jakb, Atki, Bslj -> ABts", t_iajb, T_iajb, 1 / D_iajb, F_1R[:, :, so, so], F_1R[:, :, so, so])
    + 4 * np.einsum("iajb, lajb, jakb, Atki, Bslk -> ABts", t_iajb, T_iajb, 1 / D_iajb, F_1R[:, :, so, so], F_1R[:, :, so, so])
    - 4 * np.einsum("iajb, jbkd, jakb, Atki, Bsad -> ABts", t_iajb, T_iajb, 1 / D_iajb, F_1R[:, :, so, so], F_1R[:, :, sv, sv])
    - 4 * np.einsum("iajb, jbkd, idjb, Atad, Bski -> ABts", t_iajb, T_iajb, 1 / D_iajb, F_1R[:, :, sv, sv], F_1R[:, :, so, so])
    - 4 * np.einsum("iajb, jdka, jakb, Atki, Bsbd -> ABts", t_iajb, T_iajb, 1 / D_iajb, F_1R[:, :, so, so], F_1R[:, :, sv, sv])
    - 4 * np.einsum("iajb, jdka, idja, Atbd, Bski -> ABts", t_iajb, T_iajb, 1 / D_iajb, F_1R[:, :, sv, sv], F_1R[:, :, so, so])
)

最后，我们可以验证上述计算过程的给出的正确的：

In [24]:
np.allclose(hess_mp2, hess_mp2_ref)

True

我们指出，尽管我们确实地生成了 $U_{ai}^{A_t}$，但没有生成过 Canonical HF 下不安全的 `hfh.U_1` $U_{pq}^{A_t}$ 和 `hfh.U_2_vo` $U_{ai}^{A_t B_s}$．这些不安全的矩阵都已经由旋转后安全的矩阵所替代．

在 HFHelper 类中，隐藏变量 `hfh._U_1` 与 `hfh._U_2_vo` 是实际储存一阶全 U 矩阵和二阶非占-占据的 U 矩阵的对象；我们可以发现这些变量并没有被生成；但 `hfh._U_1_vo` 作为一阶非占-占据的 U 矩阵对象是 NumPy 的向量．

In [32]:
print(type(hfh._U_1))
print(type(hfh._U_2_vo))
print(type(hfh._U_1_vo))

<class 'NoneType'>
<class 'NoneType'>
<class 'numpy.ndarray'>
